# **Hypothesis Testing**

_John Andrew Dixon_

---

##### **Imports**

---

> ##### **_Note On The Data_**
> This project states the following regarding the data to use:
> <pre>
> <b><em>Define a smaller (but logical) period of time to use for your analyses (e.g. last 10 years, 2010-2019/pre-pandemic, etc).</em></b>
> </pre>
> So, I defined my data as the first decade of the 21st century (2001-2010), and I'll only be analyzing movies from this time period.
>
> I retrieved the rest of this data within the `2a-efficient-API.ipynb` file and combined the files at the end of the `2b-EDA.ipynb`. I then reran the `3-SQL.ipynb` to add the new data to the database.

Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates

Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?